In [1]:
import pandas as pd
import string
import nltk
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [3]:
from google.colab import files
uploaded = files.upload()import os
print(os.listdir())

Saving archive (1).zip to archive (1).zip


In [2]:
df = pd.read_csv("7817.csv")

# Select required columns
df = df[['reviews.text', 'reviews.rating']]
df.columns = ['review', 'rating']

df.dropna(inplace=True)

df.head()


FileNotFoundError: [Errno 2] No such file or directory: '7817.csv'

In [5]:
!unzip archive.zip

Archive:  archive.zip
  inflating: 7817_1.csv              


In [4]:
import os
print(os.listdir())

['.config', 'archive (1).zip', 'archive.zip', 'sample_data']


In [8]:
from google.colab import files
files

<module 'google.colab.files' from '/usr/local/lib/python3.12/dist-packages/google/colab/files.py'>

In [7]:
import os
print(os.listdir('/content'))

['.config', '7817_1.csv', 'archive (1).zip', 'archive.zip', 'sample_data']


In [11]:
df = pd.read_csv("7817.csv")

df = df[['reviews.text', 'reviews.rating']]
df.columns = ['review', 'rating']
df.dropna(inplace=True)

df.head()

FileNotFoundError: [Errno 2] No such file or directory: '7817.csv'

In [12]:
import os
print(os.listdir('/content'))

['.config', '7817_1.csv', 'archive (1).zip', 'archive.zip', 'sample_data']


In [10]:
df = pd.read_csv("7817_1.csv")

In [15]:
fake = df[df['label'] == 1]
genuine = df[df['label'] == 0]

genuine_sample = genuine.sample(len(fake), random_state=42)

balanced_df = pd.concat([fake, genuine_sample])
balanced_df = balanced_df.sample(frac=1, random_state=42)

print("Balanced Distribution:")
print(balanced_df['label'].value_counts())

Balanced Distribution:
label
1    76
0    76
Name: count, dtype: int64


In [14]:
df['label'] = df['rating'].apply(lambda x: 1 if x <= 2 else 0)

print("Original Distribution:")
print(df['label'].value_counts())

Original Distribution:
label
0    1101
1      76
Name: count, dtype: int64


In [13]:
import pandas as pd

df = pd.read_csv("7817_1.csv")

df = df[['reviews.text', 'reviews.rating']]
df.columns = ['review', 'rating']

df.dropna(inplace=True)

df.head()

,review,rating
0,I initially had trouble deciding between the p...,5.0
1,Allow me to preface this with a little history...,5.0
2,I am enjoying it so far. Great for reading. Ha...,4.0
3,I bought one of the first Paperwhites and have...,5.0
4,I have to say upfront - I don't like coroporat...,5.0


In [18]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(balanced_df['cleaned'])
y = balanced_df['label']

In [20]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [21]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

Accuracy: 0.6304347826086957

Classification Report:

              precision    recall  f1-score   support

           0       0.58      0.82      0.68        22
           1       0.73      0.46      0.56        24

    accuracy                           0.63        46
   macro avg       0.66      0.64      0.62        46
weighted avg       0.66      0.63      0.62        46



In [22]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[18  4]
 [13 11]]


In [23]:
def fuzzy_decision(prob):
      if prob < 0.45:
              return "Likely Genuine"
      elif prob <= 0.55:
              return "Suspicious"
      else:
              return "Likely Fake"

In [24]:
def predict_review(review):
      cleaned = clean_text(review)
      vector = vectorizer.transform([cleaned])
      prob = model.predict_proba(vector)[0][1]
      label = fuzzy_decision(prob)
      return label, round(prob * 100, 2)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
      X, y, test_size=0.3, random_state=42
      )

In [30]:
print(predict_review("Limited offer click here to win free prize"))
print(predict_review("The product quality is excellent and delivery was fast"))

('Suspicious', np.float64(50.38))
('Suspicious', np.float64(49.06))


In [26]:
def fuzzy_decision(prob):
      if prob < 0.40:
              return "Likely Genuine"
      elif prob <= 0.60:
              return "Suspicious"
      else:
              return "Likely Fake"

In [17]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = str(text).lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

balanced_df['cleaned'] = balanced_df['review'].apply(clean_text)

balanced_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,review,rating,label,cleaned
1587,I am now having to order my third remote becau...,2.0,1,order third remote randomly dies reason replac...
177,I purchased the original Kindle Fire in 2011 a...,5.0,0,purchased original kindle fire 2011 second gen...
916,It exceeded my expectations and recommend this...,5.0,0,exceeded expectations recommend product friends
189,Not that Amazon needs another 5 star review fo...,5.0,0,amazon needs another 5 star review remote damn...
381,First I would like to say that I am coming fro...,4.0,0,first would like say coming using google tv fi...
